In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
import time
import ghalton
import numpy as np
import numpy.fft as fft
import cv2
import scipy.ndimage
import cPickle as pickle
import sys
sys.path.insert(1,'/Users/zyzdiana/GitHub/AC297r-Volume-Registration/code')
import theano
import theano.tensor as T
rng = np.random.RandomState(42)
theano.config.floatX = 'float32'

In [3]:
from utils import clean,to_radian,hann,res_to_rad
from get_data import get_data
from rotation_2d import rot_cost_func
from visualize import plot_cost_func
from cost_functions import cf_ssd
from mask import sphere_mask
from Bessel_Interp import bessel_cost_func,bessel_rotate,generate_hann_windowed_bessel_interpolation
from Bessel_Interp import hann_windowed_bessel_halton_in_circ_cost_func, bessel_halton_in_circ_cost_func

In [4]:
# load in data
data_dict_head = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/data_dict_head.p','rb'))
list_6_4mm, list_8mm, list_10mm = get_data()

### phase correlation

In [5]:
def phase_translation_2d(im0, im1):
    '''
    detect integer translation from phase translation
    '''
    f0 = fft.fft2(im0)
    f1 = fft.fft2(im1)
    ir = abs(fft.ifft2((f0 * f1.conjugate()) / (abs(f0) * abs(f1))))
    t0, t1 = np.unravel_index(np.argmax(ir), im0.shape)
    return [t0, t1]

In [6]:
# testing the 2d translation with 6.4mm resolution
im0 = fft.fft2(fft.ifft2(list_6_4mm[0].max(axis=1)))
im1 = fft.fft2(fft.ifft2(list_6_4mm[4].max(axis=1)))
print 'integer translation:',phase_translation_2d(im0, im1)

integer translation: [2, 0]


In [7]:
# testing the 2d translation with 8mm resolution
im0 = fft.fft2(fft.ifft2(list_8mm[0].max(axis=1)))
im1 = fft.fft2(fft.ifft2(list_8mm[4].max(axis=1)))
print 'integer translation:',phase_translation_2d(im0, im1)

integer translation: [1, 0]


In [8]:
# testing the 2d translation with 10mm resolution
im0 = fft.fft2(fft.ifft2(list_10mm[0].max(axis=1)))
im1 = fft.fft2(fft.ifft2(list_10mm[4].max(axis=1)))
print 'integer translation:',phase_translation_2d(im0, im1)

integer translation: [1, 0]


In [9]:
def phase_translation_3d(vol0, vol1):
    f0 = fft.fftn(vol0)
    f1 = fft.fftn(vol1)
    ir = abs(fft.ifftn((f0 * f1.conjugate()) / (abs(f0) * abs(f1))))
    t0, t1, t2 = np.unravel_index(np.argmax(ir), vol0.shape)
    return [t0, t1, t2]

In [10]:
# test the 3d translation with 8mm resolution
vol0 = fft.fft2(fft.ifft2(list_8mm[0]))
vol1 = fft.fft2(fft.ifft2(list_8mm[4]))
print phase_translation_3d(vol0,vol1)

# test the 3d translation with 6_4mm resolution
vol0 = fft.fft2(fft.ifft2(list_6_4mm[0]))
vol1 = fft.fft2(fft.ifft2(list_6_4mm[4]))
print phase_translation_3d(vol0,vol1)

# test the 3d translation with 10mm resolution
vol0 = fft.fft2(fft.ifft2(list_10mm[0]))
vol1 = fft.fft2(fft.ifft2(list_10mm[4]))
print phase_translation_3d(vol0,vol1)

[1, 0, 0]
[2, 0, 0]
[1, 0, 0]


### Gradient descent

In [11]:
from GD import gradient_descent,

In [41]:
#8mm Translation 10mm & rotation 5 degrees
testVol1 = sphere_mask(list_8mm[0],16)
testVol2 = sphere_mask(list_8mm[4],16)

R = np.zeros([3]).astype('float32')
t = np.zeros([3]).astype('float32')
errors,R_res,t_res = gradient_descent(testVol1,testVol2,R,t,0.2)

Minimized error:  0.000915302698368
Rotation matrix R:
[ 0.02171955  0.01188172  0.03826377]
Translation vector t:
[ 0.09501962 -0.94326574  0.06032803]


In [68]:
#8mm Translation 8mm & rotation 5 degrees
testVol1 = sphere_mask(list_8mm[0],16)
testVol2 = sphere_mask(list_8mm[4],16)

R = np.zeros([3]).astype('float32')
t = np.array([0,-1,0]).astype('float32')
errors,R_res,t_res = gradient_descent(testVol1,testVol2,R,t,0.2)

Minimized error:  0.000918042299288
Rotation matrix R:
[ 0.00828353  0.00440224 -0.00038155]
Translation vector t:
[ 0.08003291 -0.95458353  0.05101336]


In [50]:
#10mm Translation 10mm + rotation 5 degrees
testVol1 = sphere_mask(list_10mm[0],13)
testVol2 = sphere_mask(list_10mm[4],13)

R = np.zeros([3]).astype('float32')
t = np.zeros([3]).astype('float32')
errors,R_res,t_res = gradient_descent(testVol1,testVol2,R,t,0.2)

Minimized error:  0.000420062844774
Rotation matrix R:
[ 0.02979058 -0.00070872  0.01676924]
Translation vector t:
[-0.00521307 -0.76371765 -0.0596025 ]
